In [1]:
#Importar librerias
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D,MaxPooling2D
import os

In [2]:
#Leer la ruta de las imagenes de entrenamiento y validacion

entrenamiento = r'D:\velaz\Documents\8B\Sistemas Inteligentes\EI_Emociones\personas\train'
validacion = r'D:\velaz\Documents\8B\Sistemas Inteligentes\EI_Emociones\personas\test'

In [3]:
#Establecer los hiperparametros de la Red Neuronal Convolucional

#Definir hiperparametros de entrenamiento
epocas = 30

altura, anchura = 48,48 #tamaño de las imagenes
batch_size = 32
clases = 7

In [4]:
#Generar datos sinteticos. Son utiles cuando tenemos pocos datos de entrenamiento

entrenar = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					horizontal_flip=True,
					fill_mode='nearest')

validar = ImageDataGenerator(rescale=1./255)

#Leer las imagenes
#Entrenamiento
imagenes_entrenamiento = entrenar.flow_from_directory(
					entrenamiento,
					color_mode='grayscale',
					target_size=(altura, anchura),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)

#Validacion
imagenes_validacion = validar.flow_from_directory(
							validacion,
							color_mode='grayscale',
							target_size=(altura, anchura),
							batch_size=32,
							class_mode='categorical',
							shuffle=True)

class_labels=['Angry','Disgust', 'Fear', 'Happy','Neutral','Sad','Surprise']
img, label = imagenes_entrenamiento.__next__()


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [5]:
#Construir la arquitectura de la red neuronal convolucional
CNN = Sequential()

#Primera capa convolucional
CNN.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(altura, anchura, 1)))

#Segunda capa convolucional
CNN.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Dropout(0.1))

#Tercera capa convolucional
CNN.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Dropout(0.1))

#Cuarta capa convolucional
CNN.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
CNN.add(MaxPooling2D(pool_size=(2, 2)))
CNN.add(Dropout(0.1))

#Aplanar las matrices para conectarlas con la red densa
CNN.add(Flatten())

#Entrenar las características extraídas
#Capa oculta de la red densa
CNN.add(Dense(512, activation='relu'))
CNN.add(Dropout(0.2))

#Capa de salida con softmax para clasificación multiclase
CNN.add(Dense(clases, activation='softmax'))

C:\Users\velaz\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
#Establecer hiperparametros de entrenamiento
CNN.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumen de la arquitectura
print(CNN.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 44, 44, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │         3,591 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,489,095 (9.50 MB)

 Trainable params: 2,489,095 (9.50 MB)

 Non-trainable params: 0 (0.00 B)

None


In [7]:
train_path = r'D:\velaz\Documents\8B\Sistemas Inteligentes\EI_Emociones\personas\train'
test_path = r'D:\velaz\Documents\8B\Sistemas Inteligentes\EI_Emociones\personas\test'

num_entrenamiento = 0
for root, dirs, files in os.walk(train_path):
    num_entrenamiento += len(files)

num_validacion = 0
for root, dirs, files in os.walk(test_path):
    num_validacion += len(files)

print("Total imágenes entrenamiento:", num_entrenamiento)
print("Total imágenes validación:", num_validacion)

Total imágenes entrenamiento: 28709
Total imágenes validación: 7178


In [ ]:
#Entrenar la red neuronal convolucional
historico = CNN.fit(imagenes_entrenamiento,
                steps_per_epoch=num_entrenamiento//32,
                epochs=epocas,
                validation_data=imagenes_validacion,
                validation_steps=num_validacion//32)

In [ ]:
# Guardar modelo
CNN.save(r"D:\velaz\Documents\8B\Sistemas Inteligentes\EI_Emociones\modelo\modelo.h5")